In [1]:
import torch
import numpy as np


# Data Loading

In [2]:
df = open('./CSVs/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

In [4]:
def prepare_input(moves):
    x = torch.zeros((19,19,4) , dtype=torch.int8)
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        # last_move_column = coordinates[moves[-1][2]]
        # last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = torch.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [5]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 100160, Total Moves: 22853380


In [6]:
# x = []
# y = []

# def get_one_game(args):
#     (game_index , moves_list) = args
#     x = []
#     y = []
#     for count, move in enumerate(moves_list):
#         if (game_index + count) % 5 == 0: #% 20 == 0:
#             x.append(prepare_input(moves_list[:count]))
#             y.append(prepare_label(moves_list[count]))
#     return torch.stack(x) , torch.tensor(y)

# from concurrent.futures import ProcessPoolExecutor
# from tqdm import tqdm

# with ProcessPoolExecutor(max_workers=3) as executor:
#     outputs = executor.map(get_one_game, [(game_index , game.split(',')) for game_index , game in enumerate(games)] , chunksize=200)
#     x = []
#     y = []
#     for (output_x , output_y) in tqdm(outputs , total=n_games):
#         x.append(output_x)
#         y.append(output_y)

#     x = torch.concat(x, dim=0)
#     y = torch.concat(y, dim=0)

from tqdm import tqdm
x = []
y = []
for game_index , game in tqdm(enumerate(games) , total=n_games):
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        if (game_index + count) % 5 == 0: #% 20 == 0:
            x.append(prepare_input(moves_list[:count]))
            y.append(prepare_label(moves_list[count]))
x = torch.stack(x)
y = torch.tensor(y)

100%|██████████| 100160/100160 [54:13<00:00, 30.78it/s] 


In [7]:
x.shape

torch.Size([4570718, 19, 19, 4])

In [8]:
y.shape

torch.Size([4570718])

In [9]:
datset = torch.utils.data.TensorDataset(x , y)
train_size = int(len(datset) * .8)
valid_size = len(datset) - train_size
train_set, valid_set = torch.utils.data.random_split(datset , [train_size , valid_size] )

In [10]:
import torch.nn as nn
import math
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.0, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x:torch.nn) -> torch.nn:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x.transpose(0,1)
        x = x + self.pe[:x.size(0)]
        return self.dropout(x).transpose(0,1)

# class ResidualBlock(nn.Module):
#     def __init__(self, dim):
#         super().__init__()

#         self.cnn = nn.Sequential(
#             nn.Conv2d(dim , dim , kernel_size=3 , stride=1 , padding=1),
#             nn.Softplus(),
#             nn.Conv2d(dim , dim , kernel_size=3 , stride=1 , padding=1),
#         )
#         # self.activate = nn.Softplus()
#         self.norm = nn.InstanceNorm2d(dim , momentum=0.01 , affine=True , track_running_stats=True)

#     def forward(self , x):
#         return self.norm(x + self.cnn(x))#self.activate()
    
# class AttentionBlock(nn.Module):
#     def __init__(self, d_model):
#         super().__init__()
#         self.self_attn = nn.MultiheadAttention(d_model, num_heads=8, dropout=0, batch_first=True)
#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)

#         self.feed_forward = nn.Sequential(
#             nn.Linear(d_model , d_model * 2),
#             nn.Softplus(),
#             nn.Linear(d_model * 2, d_model),
#         )

#     def forward(self , target , memory , **kwargs):
#         target = self.norm1(target + self.self_attn(target , memory , memory , need_weights=False)[0])
#         target = self.norm2(target + self.feed_forward(target))
#         return target

class MyModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.atten_size = 512

        self.input_layer = nn.Sequential(
            nn.Linear(4 , self.atten_size),
            nn.GELU(),
        )

        self.position_encoding = PositionalEncoding(self.atten_size , dropout=0 , max_len=361)

        encoder_layer = nn.TransformerEncoderLayer(d_model=self.atten_size,  nhead=8 , dim_feedforward=2048 , batch_first=True , activation="gelu")
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=5)

        # self.output_conv = nn.Conv1d(self.atten_size , 1 , kernel_size=1 , stride=1 , padding=0)
        self.output_linear = nn.Linear(self.atten_size , 1)

    def forward(self , x):
        # [batch , w , h , dim]
        x = self.input_layer(x).flatten(1,2)

        # [batch , w * h , dim]
        x = self.position_encoding(x)
        x = self.transformer_encoder(x)

        # x = x.transpose(1 , 2)
        # [batch , dim , w * h]
        # x = self.output_conv(x).squeeze(1)
        # [batch , w * h]

        return self.output_linear(x).squeeze(-1)
    

In [11]:
from tqdm import tqdm
from IPython.display import clear_output

model = MyModel()

device = torch.device("cuda")
model.to(device)
# model.load_state_dict(torch.load("model/play_dan_xxlarge_backup.pt") , strict=False)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scheduler =  torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.93)
scaler = torch.cuda.amp.GradScaler()

train_dataloader = torch.utils.data.DataLoader(train_set , batch_size=128 , shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_set , batch_size=128)

loss_fn = torch.nn.CrossEntropyLoss()

from sklearn.metrics import f1_score , accuracy_score

dict_state = {
    "train progress" : "0/0",
    "valid progress" : "0/0",

    "train_loss" : "na",
    "valid_loss" : "na",
    "valid_f1" : "na",
    "valid_accuracy" : "na",
}

with(tqdm(range(100), desc="Epochs") as pbar) :

    dict_state.update({
        "train progress" : f"0/{len(train_dataloader)}",
        "valid progress" : f"0/{len(valid_dataloader)}",
        })
    pbar.set_postfix(dict_state , refresh=True)

    best_loss = 100
    for i in pbar:
        train_loss = 0
        valid_loss = 0

        model.train()
        for i , (x , y) in enumerate(train_dataloader):
            optimizer.zero_grad()
            x = x.to(device).float()
            y = y.to(device)
            with torch.cuda.amp.autocast(dtype=torch.float16 , enabled=True):
                output = model(x)

                loss = loss_fn(output , y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            dict_state.update({"train progress" : f"{i+1}/{len(train_dataloader)}"})
            pbar.set_postfix(dict_state , refresh=(i % 5) == 0)

        scheduler.step()
        
        metrics_true = []
        metrics_pred = []
        model.eval()
        with torch.no_grad():
            for i , (x , y) in enumerate(valid_dataloader):
                x = x.to(device).float()
                y = y.to(device)
                with torch.cuda.amp.autocast(dtype=torch.float16 , enabled=True):
                    output = model(x)
                    loss = loss_fn(output , y)
                scaler.scale(loss)
                valid_loss += loss.item()
                
                metrics_true.extend(y.cpu().flatten().tolist())
                metrics_pred.extend(torch.argmax(output , dim=-1).cpu().flatten().tolist())
                dict_state.update({"valid progress" : f"{i+1}/{len(valid_dataloader)}"})
                pbar.set_postfix(dict_state , refresh=(i % 10) == 0)

        train_loss /= len(train_dataloader)
        valid_loss /= len(valid_dataloader)
        
        valid_f_score = f1_score(metrics_true , metrics_pred , average='macro')
        valid_accuracy = accuracy_score(metrics_true , metrics_pred)
        
        dict_state.update({"train_loss":train_loss , "valid_loss":valid_loss , "valid_f1": valid_f_score , "valid_accuracy":valid_accuracy})
        pbar.set_postfix(dict_state , refresh=True)
        
        if best_loss > valid_loss:
            best_loss = valid_loss
            torch.save(model.state_dict() , "model/play_dan_xxlarge.pt")

Epochs:  23%|██▎       | 23/100 [90:31:51<301:11:11, 14081.44s/it, train progress=971/28567, valid progress=7142/7142, train_loss=1.92, valid_loss=1.87, valid_f1=0.503, valid_accuracy=0.498]  